In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("fill-mask", model="Sakonii/distilbert-base-nepali")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/559 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/105 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("Sakonii/distilbert-base-nepali")
model = AutoModelForMaskedLM.from_pretrained("Sakonii/distilbert-base-nepali")

Loading weights:   0%|          | 0/105 [00:00<?, ?it/s]

In [3]:
# Why: pipeline() is HF's easy way to use models without low-level code.
from transformers import pipeline
import gradio as gr

In [4]:
from huggingface_hub import login
login()

In [5]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration

model_id = "openai/whisper-small"

processor = WhisperProcessor.from_pretrained(model_id)
model = WhisperForConditionalGeneration.from_pretrained(model_id)
model = model.to("cuda")

preprocessor_config.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/479 [00:00<?, ?it/s]

generation_config.json: 0.00B [00:00, ?B/s]

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
from google.colab import userdata
userdata.get('STT')

In [ ]:
from transformers import pipeline
from google.colab import userdata

# Load HF token safely
try:
    hf_token = userdata.get("HF_TOKEN")
    print("HF token loaded successfully")
except Exception:
    hf_token = None
    print("WARNING: HF_TOKEN not found. Only public models will work.")

# USE A VERIFIED PUBLIC MODEL
whisper_model = pipeline(
    task="automatic-speech-recognition",
    model="openai/whisper-small",
    device=0,               # GPU
    token=hf_token
)

print("Model loaded successfully! Ready to test.")


In [ ]:
import soundfile as sf
import subprocess
import os
import numpy as np # Import numpy
from transformers import WhisperProcessor, WhisperForConditionalGeneration # Ensure these are imported if not globally available, though they should be from NH5dsO5uIfUK
import torch

audio_path = "/content/drive/MyDrive/Sarkari Sarathi /mero desh.wav"
wav_audio_path = "/content/drive/MyDrive/Sarkari Sarathi /mero_desh_converted.wav"

# Check if the file exists
if not os.path.exists(audio_path):
    print(f"Error: Audio file not found at {audio_path}. Please check the path and try again.")
else:
    print(f"Original audio file: {audio_path}")

    # Convert the audio file to WAV using ffmpeg, forcing mono output
    try:
        print(f"Converting '{audio_path}' to mono WAV format at '{wav_audio_path}'...")
        # Added -ac 1 to force mono conversion
        command = ["ffmpeg", "-i", audio_path, "-acodec", "pcm_s16le", "-ar", "16000", "-ac", "1", wav_audio_path, "-y"]
        # -y option to overwrite output file without asking
        process = subprocess.run(command, capture_output=True, text=True, check=True)
        print("FFmpeg conversion stdout:", process.stdout)
        print("FFmpeg conversion stderr:", process.stderr)
        print("Conversion successful!")

        # Now attempt to process the converted WAV file
        audio, sampling_rate = sf.read(wav_audio_path)

        # Ensure audio data is float32, which is often expected by models
        if audio.dtype != np.float32:
            audio = audio.astype(np.float32)
            print(f"Converted audio dtype to: {audio.dtype}")

        # Debugging: check audio array properties
        print(f"Audio array shape: {audio.shape}")
        print(f"Audio array dtype: {audio.dtype}")
        print(f"Sampling rate: {sampling_rate}")
        if np.isnan(audio).any() or np.isinf(audio).any():
            print("Warning: Audio array contains NaN or Inf values!")
        if audio.size == 0:
            print("Error: Audio array is empty!")
            raise ValueError("Audio array is empty after conversion and reading.")

        # Use processor and model directly instead of pipeline
        # 'processor' and 'model' should be available from NH5dsO5uIfUK
        # (Re-initializing here just for clarity in the single cell, but assuming they exist)
        model_id = "openai/whisper-small"
        processor = WhisperProcessor.from_pretrained(model_id)
        model = WhisperForConditionalGeneration.from_pretrained(model_id)

        # Check if GPU is available and move model to GPU
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model = model.to(device)

        input_features = processor(audio, sampling_rate=sampling_rate, return_tensors="pt").input_features
        input_features = input_features.to(device) # Move features to GPU if model is on GPU

        predicted_ids = model.generate(input_features)
        transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

        print("Transcription:")
        print(transcription)
    except subprocess.CalledProcessError as e:
        print(f"FFmpeg conversion failed with error: {e}")
        print(f"Stderr: {e.stderr}")
        print("Please ensure the input file is not corrupted and ffmpeg is correctly installed.")
    except FileNotFoundError:
        print("Error: ffmpeg command not found. Please ensure ffmpeg is installed (e.g., !apt-get install ffmpeg). ")
    except Exception as e:
        print(f"\nAn error occurred during audio processing after conversion: {e}")
        print(f"Type of error: {type(e)}") # Add this to clarify if it's a KeyError or other.
        print("Please ensure the converted audio file is valid and in a supported format.")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import soundfile as sf
import subprocess
import os
import numpy as np

audio_path = "/content/drive/MyDrive/Sarkari Sarathi/mero desh.wav"
wav_audio_path = "/content/drive/MyDrive/Sarkari Sarathi/mero_desh_converted.wav"

# Check if the file exists
if not os.path.exists(audio_path):
    print(f"Error: Audio file not found at {audio_path}. Please check the path and try again.")
else:
    print(f"Original audio file: {audio_path}")

    # Convert the audio file to WAV using ffmpeg, forcing mono output
    try:
        print(f"Converting '{audio_path}' to mono WAV format at '{wav_audio_path}'...")
        command = ["ffmpeg", "-i", audio_path, "-acodec", "pcm_s16le", "-ar", "16000", "-ac", "1", wav_audio_path, "-y"]
        process = subprocess.run(command, capture_output=True, text=True, check=True)
        print("FFmpeg conversion stdout:", process.stdout)
        print("FFmpeg conversion stderr:", process.stderr)
        print("Conversion successful!")

        # Now attempt to process the converted WAV file
        audio, sampling_rate = sf.read(wav_audio_path)

        # Ensure audio data is float32
        if audio.dtype != np.float32:
            audio = audio.astype(np.float32)
            print(f"Converted audio dtype to: {audio.dtype}")

        # Debugging: check audio array properties
        print(f"Audio array shape: {audio.shape}")
        print(f"Audio array dtype: {audio.dtype}")
        print(f"Sampling rate: {sampling_rate}")
        if np.isnan(audio).any() or np.isinf(audio).any():
            print("Warning: Audio array contains NaN or Inf values!")
        if audio.size == 0:
            print("Error: Audio array is empty!")
            raise ValueError("Audio array is empty after conversion and reading.")

        result = whisper_model({"sampling_rate": sampling_rate, "array": audio})
        print("Transcription:")
        print(result["text"])
    except subprocess.CalledProcessError as e:
        print(f"FFmpeg conversion failed with error: {e}")
        print(f"Stderr: {e.stderr}")
        print("Please ensure the input file is not corrupted and ffmpeg is correctly installed.")
    except FileNotFoundError:
        print("Error: ffmpeg command not found. Please ensure ffmpeg is installed (e.g., !apt-get install ffmpeg). ")
    except Exception as e:
        print(f"\nAn error occurred during audio processing after conversion: {e}")
        print(f"Type of error: {type(e)}")
        print("Please ensure the converted audio file is valid and in a supported format.")


In [ ]:
import soundfile as sf
import os
import numpy as np
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# Use the path to the successfully converted audio file
audio_file = "/content/drive/MyDrive/Sarkari Sarathi /mero_desh_converted.wav"

# Check if the converted file exists
if not os.path.exists(audio_file):
    print(f"Error: Converted audio file not found at {audio_file}. Please ensure the conversion step was successful.")
else:
    # Load audio manually
    audio, sr = sf.read(audio_file, dtype="float32")

    # Debugging: ensure audio is float32 and check properties
    if audio.dtype != np.float32:
        audio = audio.astype(np.float32)
        print(f"Converted audio dtype to: {audio.dtype}")
    print(f"Audio array shape: {audio.shape}")
    print(f"Audio array dtype: {audio.dtype}")
    print(f"Sampling rate: {sr}")

    try:
        # Use processor and model directly instead of pipeline
        model_id = "openai/whisper-small"
        processor = WhisperProcessor.from_pretrained(model_id)
        model = WhisperForConditionalGeneration.from_pretrained(model_id)

        # Check if GPU is available and move model to GPU
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model = model.to(device)

        input_features = processor(audio, sampling_rate=sr, return_tensors="pt").input_features
        input_features = input_features.to(device) # Move features to GPU if model is on GPU

        predicted_ids = model.generate(input_features)
        transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

        print("Transcription:")
        print(transcription)
    except Exception as e:
        print(f"An error occurred during audio processing: {e}")
        print(f"Type of error: {type(e)}")
        print("Please ensure the converted audio file is valid and in a supported format.")


In [ ]:
!ffmpeg -y \
-i "/content/drive/MyDrive/Sarkari Sarathi/mero desh.wav" \
-ar 16000 -ac 1 -c:a pcm_s16le \
"/content/drive/MyDrive/Sarkari Sarathi/mero_desh_fixed.wav"


In [ ]:
import os

directory_path = '/content/drive/MyDrive/Sarkari Sarathi /' # Added trailing space based on user feedback

if os.path.exists(directory_path):
    print(f"Listing contents of '{directory_path}':")
    for item in os.listdir(directory_path):
        print(item)
else:
    print(f"Error: Directory not found at {directory_path}")

In [ ]:
import soundfile as sf
import os
import numpy as np
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# --- IMPORTANT: Change this to the path of your audio file --- #
audio_file = "/content/drive/MyDrive/Sarkari Sarathi /mero desh.wav" # Example: "/content/drive/MyDrive/Sarkari Sarathi /your_audio_file.wav"

# Define a path for the converted WAV file, if conversion is needed
# This creates a new file name by adding '_converted' before the extension
base, ext = os.path.splitext(audio_file)
wav_audio_path = f"{base}_converted.wav"

# Check if the file exists
if not os.path.exists(audio_file):
    print(f"Error: Audio file not found at {audio_file}. Please check the path and try again.")
else:
    print(f"Original audio file: {audio_file}")

    try:
        # Convert the audio file to WAV using ffmpeg, forcing mono output
        print(f"Attempting to convert '{audio_file}' to mono WAV format at '{wav_audio_path}'...")
        command = ["ffmpeg", "-i", audio_file, "-acodec", "pcm_s16le", "-ar", "16000", "-ac", "1", wav_audio_path, "-y"]
        process = subprocess.run(command, capture_output=True, text=True, check=True)
        print("FFmpeg conversion stdout:", process.stdout)
        print("FFmpeg conversion stderr:", process.stderr)
        print("Conversion successful!")

        # Load the converted audio file
        audio, sr = sf.read(wav_audio_path, dtype="float32")

        # Debugging: ensure audio is float32 and check properties
        if audio.dtype != np.float32:
            audio = audio.astype(np.float32)
            print(f"Converted audio dtype to: {audio.dtype}")
        print(f"Audio array shape: {audio.shape}")
        print(f"Audio array dtype: {audio.dtype}")
        print(f"Sampling rate: {sr}")

        # Use processor and model directly
        model_id = "openai/whisper-small"
        processor = WhisperProcessor.from_pretrained(model_id)
        model = WhisperForConditionalGeneration.from_pretrained(model_id)

        # Check if GPU is available and move model to GPU
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model = model.to(device)

        input_features = processor(audio, sampling_rate=sr, return_tensors="pt").input_features
        input_features = input_features.to(device) # Move features to GPU if model is on GPU

        predicted_ids = model.generate(input_features)
        transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

        print("Transcription:")
        print(transcription)
    except subprocess.CalledProcessError as e:
        print(f"FFmpeg conversion failed with error: {e}")
        print(f"Stderr: {e.stderr}")
        print("Please ensure the input file is not corrupted and ffmpeg is correctly installed.")
    except FileNotFoundError:
        print("Error: ffmpeg command not found. Please ensure ffmpeg is installed (e.g., !apt-get install ffmpeg).")
    except Exception as e:
        print(f"An error occurred during audio processing: {e}")
        print(f"Type of error: {type(e)}")
        print("Please ensure the audio file is valid and in a supported format.")